In [1]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install -qU "langchain[google-genai]" --use-deprecated=legacy-resolver

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install -qU langchain-chroma --use-deprecated=legacy-resolver

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [6]:
#force downgrade urllib due to errors with modernized version
%pip install urllib3==1.26.18

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install -qU langchain_community pymupdf

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip install -qU pytesseract

Note: you may need to restart the kernel to use updated packages.


In [9]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter Langsmith API Key")

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

print("YAY")

YAY


In [11]:
#Load papers 
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "dnutg/paper1.pdf"
loader = PyMuPDFLoader(file_path, mode="page", 
                        extract_tables="markdown",
                      )
docs = loader.load()

In [12]:
print(docs[2])

page_content='JAW 
Vol. 2x. No. h 
T?XlJl 
Et- Al.. 
Ncwmhcr 
IS. IWh:ISW-Jh 
1541 
DEl‘ERMINANlS 
OF HEART 
RAIlI 
VARlAHIl.IT1 
Table 1. Pratwn Correlations .1mong Measures of Heart Rate \‘ariahi!ity’ 
- 
In 
In 
In 
In 
In 
LF, HF 
In 2-h 
In 
In 
VLF+ 
LF 
HF 
TPt 
R&l 
SDNN 
pSNW 
r-MSSD 
Mean 
7.37 
h.U 
5.3 I 
7.3' I- 
I.12 
4.41 
I .50 
3. !fl 
SD 
In VLFt 
in LF 
In HF 
In TPt 
In LFlHF ruin 
In l-h SDNN 
In pNNS0 
0.7.l 
0.00 
(I.‘:3 
o.xl 
l1.51 
0.31 
IX 
I I.41 
0.M 
0.7ll 
I l.(ti 
II.:? 
Il.dU 
N’ll 
Il.hs 
IuiJ 
NYC 
ll.I!* 
ON 
(I.73 
0.71 
11.55 
- II.32 
0.7.5 
O.Vl 
11.43 
Il.11 
il.SY 
11.7' 
(I.iS 
(l.U7 
-0.3 
-- NJI 
11.70 
MY 
11.93 
Statistical 
methods. 
Measures of heart rate variability 
were natural log transformed because their distributions were 
highly skewed. Associations among transformed variables were 
assessed with Pearson’s correlation coefficients. In preliminary 
analyses, mean values of heart rate variability measures were 
plotted against age 

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, #chunk size
    chunk_overlap=200, #chunk overlap
    add_start_index=True, #track index
)
all_splits = text_splitter.split_documents(docs)
print(f"Split dem cheeks boi (into a diclosed number of subdocs): {len(all_splits)} sub-documents.")

Split dem cheeks boi (into a diclosed number of subdocs): 48 sub-documents.


In [14]:
document_ids = vector_store.add_documents(documents=all_splits)
print(document_ids[:3])

['ee1c3d13-d0f7-484c-b967-aee720edbc8c', '2be772c6-a230-46aa-8492-83ee1d3b4ad8', '8fb8c3f4-3f57-413a-a347-6cda8583aee6']


In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompt_template.invoke({"topic":"cats"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me a joke about cats', additional_kwargs={}, response_metadata={})])